In [4]:
import os

import pandas as pd

from common.libs.data_analysis.SeriesDescriber import *
from common.libs.data_analysis.SeriesDescriberWRTTarget import *
from common.libs.DataDisplayer import *

In [5]:
TRAIN_FILE = os.path.join(os.getcwd(),
                          "inputs",
                          "model_ready",
                          "train",
                          "0.csv")

TEST_FILE = os.path.join(os.getcwd(),
                         "inputs",
                         "model_ready",
                         "test",
                         "0.csv")

In [6]:
train_df = pd.read_csv(TRAIN_FILE)
test_df = pd.read_csv(TEST_FILE)

In [7]:
def df_info(df):
    display_side_by_side(df.describe(), df.head())
    df.info(verbose=1, show_counts=True)

In [8]:
df_info(train_df)

Unnamed: 0 
 store_nbr 
 sales 
 onpromotion 
 dcoilwtico 
 cluster 
 vacation holiday 
 vacation additional 
 vacation bridge 
 vacation work day 
 vacation event 
 
 
 
 
 count 
 5.557200e+04 
 55572.0 
 55572.000000 
 55572.000000 
 55572.000000 
 55572.0 
 55572.000000 
 55572.000000 
 55572.000000 
 55572.000000 
 55572.000000 
 
 
 mean 
 1.499569e+06 
 1.0 
 254.534889 
 2.483625 
 67.894072 
 13.0 
 0.030285 
 0.019596 
 0.001781 
 0.002969 
 0.032660 
 
 
 std 
 8.662894e+05 
 0.0 
 596.827146 
 12.713831 
 25.659571 
 0.0 
 0.171372 
 0.138609 
 0.042170 
 0.054409 
 0.177748 
 
 
 min 
 0.000000e+00 
 1.0 
 0.000000 
 0.000000 
 26.190000 
 13.0 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 25% 
 7.497845e+05 
 1.0 
 1.000000 
 0.000000 
 46.395938 
 13.0 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 50% 
 1.499569e+06 
 1.0 
 19.000000 
 0.000000 
 53.255115 
 13.0 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 75% 
 2.249354e+06 
 1.0 
 166.000000 
 0.000000 
 95.682143 
 13.0 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 max 
 2.999138e+06 
 1.0 
 9065.000000 
 227.000000 
 110.620000 
 13.0 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 
 
 
 
 
 
 Unnamed: 0 
 date 
 store_nbr 
 family 
 sales 
 onpromotion 
 dcoilwtico 
 city 
 state 
 type 
 cluster 
 vacation holiday 
 vacation additional 
 vacation bridge 
 vacation work day 
 vacation event 
 
 
 
 
 0 
 0 
 2013-01-01 
 1 
 AUTOMOTIVE 
 0.0 
 0 
 93.076667 
 Quito 
 Pichincha 
 D 
 13 
 1 
 0 
 0 
 0 
 0 
 
 
 1 
 1 
 2013-01-01 
 1 
 BABY CARE 
 0.0 
 0 
 93.076667 
 Quito 
 Pichincha 
 D 
 13 
 1 
 0 
 0 
 0 
 0 
 
 
 2 
 2 
 2013-01-01 
 1 
 BEAUTY 
 0.0 
 0 
 93.076667 
 Quito 
 Pichincha 
 D 
 13 
 1 
 0 
 0 
 0 
 0 
 
 
 3 
 3 
 2013-01-01 
 1 
 BEVERAGES 
 0.0 
 0 
 93.076667 
 Quito 
 Pichincha 
 D 
 13 
 1 
 0 
 0 
 0 
 0 
 
 
 4 
 4 
 2013-01-01 
 1 
 BOOKS 
 0.0 
 0 
 93.076667 
 Quito 
 Pichincha 
 D 
 13 
 1 
 0 
 0 
 0 
 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55572 entries, 0 to 55571
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           55572 non-null  int64  
 1   date                 55572 non-null  object 
 2   store_nbr            55572 non-null  int64  
 3   family               55572 non-null  object 
 4   sales                55572 non-null  float64
 5   onpromotion          55572 non-null  int64  
 6   dcoilwtico           55572 non-null  float64
 7   city                 55572 non-null  object 
 8   state                55572 non-null  object 
 9   type                 55572 non-null  object 
 10  cluster              55572 non-null  int64  
 11  vacation holiday     55572 non-null  int64  
 12  vacation additional  55572 non-null  int64  
 13  vacation bridge      55572 non-null  int64  
 14  vacation work day    55572 non-null  int64  
 15  vacation event       55572 non-null 

In [9]:
df_info(test_df)

Unnamed: 0 
 store_nbr 
 onpromotion 
 dcoilwtico 
 cluster 
 vacation holiday 
 vacation additional 
 vacation bridge 
 vacation work day 
 vacation event 
 
 
 
 
 count 
 528.000000 
 528.0 
 528.000000 
 528.000000 
 528.0 
 528.0 
 528.0 
 528.0 
 528.0 
 528.0 
 
 
 mean 
 13381.000000 
 1.0 
 5.950758 
 47.277760 
 13.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 std 
 8222.409693 
 0.0 
 19.595178 
 0.667026 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 min 
 0.000000 
 1.0 
 0.000000 
 45.960000 
 13.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 25% 
 6690.500000 
 1.0 
 0.000000 
 47.002500 
 13.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 50% 
 13381.000000 
 1.0 
 0.000000 
 47.250000 
 13.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 75% 
 20071.500000 
 1.0 
 5.000000 
 47.641562 
 13.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 max 
 26762.000000 
 1.0 
 256.000000 
 48.590000 
 13.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 
 
 
 
 Unnamed: 0 
 date 
 store_nbr 
 family 
 onpromotion 
 dcoilwtico 
 city 
 state 
 type 
 cluster 
 vacation holiday 
 vacation additional 
 vacation bridge 
 vacation work day 
 vacation event 
 
 
 
 
 0 
 0 
 2017-08-16 
 1 
 AUTOMOTIVE 
 0 
 46.8 
 Quito 
 Pichincha 
 D 
 13 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 1 
 2017-08-16 
 1 
 BABY CARE 
 0 
 46.8 
 Quito 
 Pichincha 
 D 
 13 
 0 
 0 
 0 
 0 
 0 
 
 
 2 
 2 
 2017-08-16 
 1 
 BEAUTY 
 2 
 46.8 
 Quito 
 Pichincha 
 D 
 13 
 0 
 0 
 0 
 0 
 0 
 
 
 3 
 3 
 2017-08-16 
 1 
 BEVERAGES 
 20 
 46.8 
 Quito 
 Pichincha 
 D 
 13 
 0 
 0 
 0 
 0 
 0 
 
 
 4 
 4 
 2017-08-16 
 1 
 BOOKS 
 0 
 46.8 
 Quito 
 Pichincha 
 D 
 13 
 0 
 0 
 0 
 0 
 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           528 non-null    int64  
 1   date                 528 non-null    object 
 2   store_nbr            528 non-null    int64  
 3   family               528 non-null    object 
 4   onpromotion          528 non-null    int64  
 5   dcoilwtico           528 non-null    float64
 6   city                 528 non-null    object 
 7   state                528 non-null    object 
 8   type                 528 non-null    object 
 9   cluster              528 non-null    int64  
 10  vacation holiday     528 non-null    int64  
 11  vacation additional  528 non-null    int64  
 12  vacation bridge      528 non-null    int64  
 13  vacation work day    528 non-null    int64  
 14  vacation event       528 non-null    int64  
dtypes: float64(1), int64(9), object(5)
memor

In [12]:
pd.value_counts(train_df['family'])

C:\Users\zuolo\AppData\Local\Temp\ipykernel_4084\1857214588.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(train_df['date'])


date
2013-01-01    33
2013-01-02    33
2013-01-03    33
2013-01-04    33
2013-01-05    33
              ..
2017-08-11    33
2017-08-12    33
2017-08-13    33
2017-08-14    33
2017-08-15    33
Name: count, Length: 1684, dtype: int64

In [13]:
pd.value_counts(test_df['family'])

C:\Users\zuolo\AppData\Local\Temp\ipykernel_4084\2174364582.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(test_df['date'])


date
2017-08-16    33
2017-08-17    33
2017-08-18    33
2017-08-19    33
2017-08-20    33
2017-08-21    33
2017-08-22    33
2017-08-23    33
2017-08-24    33
2017-08-25    33
2017-08-26    33
2017-08-27    33
2017-08-28    33
2017-08-29    33
2017-08-30    33
2017-08-31    33
Name: count, dtype: int64

In [16]:
df_pivot = train_df.pivot(index='date', columns='family', values='sales')

# Rename columns to include "_Cost"
df_pivot.columns = [f"{col}_Cost" for col in df_pivot.columns]

# train_df['family'] = df_pivot


In [17]:
df_pivot

,AUTOMOTIVE_Cost,BABY CARE_Cost,BEAUTY_Cost,BEVERAGES_Cost,BOOKS_Cost,BREAD/BAKERY_Cost,CELEBRATION_Cost,CLEANING_Cost,DAIRY_Cost,DELI_Cost,...,MAGAZINES_Cost,MEATS_Cost,PERSONAL CARE_Cost,PET SUPPLIES_Cost,PLAYERS AND ELECTRONICS_Cost,POULTRY_Cost,PREPARED FOODS_Cost,PRODUCE_Cost,SCHOOL AND OFFICE SUPPLIES_Cost,SEAFOOD_Cost
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000,...,0.0,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.000,0.0,0.000000
2013-01-02,2.0,0.0,2.0,1091.0,0.0,470.65200,0.0,1060.0,579.0,164.069,...,0.0,369.10100,194.0,0.0,0.0,247.29700,47.000000,0.000,0.0,38.029000
2013-01-03,3.0,0.0,0.0,919.0,0.0,310.65500,0.0,836.0,453.0,151.582,...,0.0,272.31900,153.0,0.0,0.0,187.27800,63.000000,0.000,0.0,17.366001
2013-01-04,3.0,0.0,3.0,953.0,0.0,198.36600,0.0,827.0,460.0,131.411,...,0.0,454.17200,88.0,0.0,0.0,258.02300,67.000000,0.000,0.0,29.907001
2013-01-05,5.0,0.0,3.0,1160.0,0.0,301.05700,0.0,811.0,464.0,118.613,...,0.0,328.94000,141.0,0.0,0.0,212.33301,66.000000,0.000,0.0,24.842000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,0.0,1.0,1006.0,0.0,145.60700,4.0,341.0,343.0,64.302,...,4.0,385.99402,53.0,5.0,3.0,259.11800,24.963001,1115.334,0.0,19.424000
2017-08-12,6.0,0.0,3.0,1659.0,0.0,243.22000,3.0,351.0,526.0,99.488,...,6.0,211.75600,227.0,3.0,6.0,217.66400,48.058000,1762.493,0.0,20.150000
2017-08-13,1.0,0.0,1.0,803.0,0.0,136.67900,1.0,169.0,266.0,47.770,...,4.0,88.18200,45.0,2.0,0.0,115.75800,20.346000,986.669,0.0,11.378000
